# A DATA ANALYSIS FOR MICROSOFT MOVIE STUDIO
## Goals:
    1. Prepare the provide data
    2. Do analysis on the data
    3. Generate insights 

In [4]:
#importing the required libraries
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import sqlite3
import string
import random

In [5]:
#1. opening bom.movie_gross file and creating a dataframe
bom_movie_gross = pd.read_csv('.data/bom.movie_gross.csv')

#2. opening rt.movie_info file and creating a dataframe
rt_movie_info = pd.read_csv('.data/rt.movie_info.tsv', sep = '\t', index_col=0)

#3. opening rt.reviews file and creating a dataframe
rt_reviews = pd.read_csv('.data/rt.reviews.tsv', encoding = 'unicode_escape', sep = '\t', index_col=0)

#4. opening tmdb.movies file and creating a dataframe
tmdb_movies = pd.read_csv('.data/tmdb.movies.csv', index_col=0)

#5. opening tn.movie_budgets file and creating a dataframe
tn_movie_budgets = pd.read_csv('.data/tn.movie_budgets.csv', index_col=0)

#6. opening imdb file and creating a dataframe
    #query the database and join the tables movie basics and movie ratings
conn = sqlite3.connect('.data/im.db')


#### Function for checking various information about a dataframe

In [6]:
def df_info(df):
    df_shape = df.shape
    missing_sum = df.isna().sum()
    names = df.columns
    info = df.info()
    print('\033[95m' + f'Column names: {names}' +  '\033[0m')
    print(f'Shape: {df_shape}')
    print(f'Df info: {info}')
    print('\033[94m' + 'Missing sum:' + '\033[0m')
    print('\033[94m' + f' {missing_sum}' + '\033[0m')
    return

### Joining movie basics table with movie ratings and movie akas tables

In [7]:
#joining tables and creating a dataframe
imdb = pd.read_sql('''SELECT *
                      FROM movie_basics
                      JOIN ( SELECT averagerating, numvotes, region, movie_id
                             FROM movie_ratings
                             JOIN movie_akas
                             USING(movie_id)
                           ) ratings
                       USING(movie_id)
''', conn)

#### cleaning the dataframe formed after joining the tables

In [8]:
imdb

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,region
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,None
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,IN
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,IN
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,IN
4,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,IN
...,...,...,...,...,...,...,...,...,...
261801,tt9905462,Pengalila,Pengalila,2019,111.0,Drama,8.4,600,IN
261802,tt9905462,Pengalila,Pengalila,2019,111.0,Drama,8.4,600,IN
261803,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130.0,Drama,8.4,365,None
261804,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130.0,Drama,8.4,365,IN


In [9]:
#removing duplicated rows
imdb = imdb.drop_duplicates()

#removing null rows in region rows with the string none
imdb = imdb[(imdb['region'].notna()) & (imdb['region'] != 'None')]

#drop duplicated rows while merging the region values
imdb = imdb.groupby('movie_id', as_index=False).agg({'primary_title': 'first',
                                                  'original_title': 'first',
                                                  'start_year': 'first',
                                                  'runtime_minutes': 'first',
                                                  'genres': 'first',
                                                  'averagerating': 'first',
                                                  'numvotes': 'first',
                                                  'region': ', '.join})

imdb

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,region
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,IN
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,"IN, XWW"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,"AR, BR, DE, ES, FR, GB, IT, PL, PT, RU, US, VE"
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,IN
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,"CL, PL, XWW"
...,...,...,...,...,...,...,...,...,...
69547,tt9899860,Watching This Movie Is a Crime,Didan in film jorm ast,2019,100.0,"Drama,Thriller",8.1,7,"IR, XWW"
69548,tt9899880,Columbus,Columbus,2018,85.0,Comedy,5.8,5,IR
69549,tt9903952,BADMEN with a good behavior,BADMEN with a good behavior,2018,87.0,"Comedy,Horror",9.2,5,DE
69550,tt9905462,Pengalila,Pengalila,2019,111.0,Drama,8.4,600,IN


In [10]:
#checking which columns have missing data
imdb.isna().sum()

movie_id              0
primary_title         0
original_title        0
start_year            0
runtime_minutes    6741
genres              637
averagerating         0
numvotes              0
region                0
dtype: int64

In [11]:
#checking missing percentage
missing_percentage_genres = imdb.genres.isna().sum() * 100 / len(imdb.genres)
missing_percentage_minutes = imdb.runtime_minutes.isna().sum() * 100 / len(imdb.runtime_minutes)
print('Missing data for genres', format(missing_percentage_genres, '.2f'))
print('Missing data for runtime', format(missing_percentage_minutes, '.2f'))

Missing data for genres 0.92
Missing data for runtime 9.69


In [12]:
#removing all rows in genre and runtime minutes as their percentage is negligible
imdb['genres'] = imdb.genres.dropna()
imdb['runtime_minutes'] = imdb.runtime_minutes.dropna()
imdb = imdb.dropna(subset=['genres', 'runtime_minutes'])

#dropping a column
imdb = imdb.drop('primary_title', axis=1)
imdb

,movie_id,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,region
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,IN
1,tt0066787,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,"IN, XWW"
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,"AR, BR, DE, ES, FR, GB, IT, PL, PT, RU, US, VE"
4,tt0100275,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,"CL, PL, XWW"
6,tt0137204,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",8.1,263,CA
...,...,...,...,...,...,...,...,...
69547,tt9899860,Didan in film jorm ast,2019,100.0,"Drama,Thriller",8.1,7,"IR, XWW"
69548,tt9899880,Columbus,2018,85.0,Comedy,5.8,5,IR
69549,tt9903952,BADMEN with a good behavior,2018,87.0,"Comedy,Horror",9.2,5,DE
69550,tt9905462,Pengalila,2019,111.0,Drama,8.4,600,IN


In [13]:
imdb =imdb.rename(columns={'original_title': 'movie'})

In [14]:
#confirming if there are missing values
imdb.isna().sum()

movie_id           0
movie              0
start_year         0
runtime_minutes    0
genres             0
averagerating      0
numvotes           0
region             0
dtype: int64

In [15]:
df_info(imdb)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62374 entries, 0 to 69551
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         62374 non-null  object 
 1   movie            62374 non-null  object 
 2   start_year       62374 non-null  int64  
 3   runtime_minutes  62374 non-null  float64
 4   genres           62374 non-null  object 
 5   averagerating    62374 non-null  float64
 6   numvotes         62374 non-null  int64  
 7   region           62374 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 4.3+ MB
Column names: Index(['movie_id', 'movie', 'start_year', 'runtime_minutes', 'genres',
       'averagerating', 'numvotes', 'region'],
      dtype='object')
Shape: (62374, 8)
Df info: None
Missing sum:
 movie_id           0
movie              0
start_year         0
runtime_minutes    0
genres             0
averagerating      0
numvotes           0
region             0
dt

### Merging and cleaning bom movie gross and tn mov ie budgets data

In [16]:
#viewing information about the dataframe
tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 1 to 82
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   object
 3   domestic_gross     5782 non-null   object
 4   worldwide_gross    5782 non-null   object
dtypes: object(5)
memory usage: 271.0+ KB


In [17]:
#viewing information about the dataframe
bom_movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [18]:
#renaming a column in the dataframe
bom_movie_gross = bom_movie_gross.rename(columns={'title': 'movie'})

In [19]:
#viewing the dataframe
tn_movie_budgets.head(4)

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"


In [20]:
#merging bom gross and tn budgets
bom_and_tn = pd.merge(bom_movie_gross, tn_movie_budgets, on='movie', how='outer')

In [21]:
bom_and_tn.tail()

,movie,studio,domestic_gross_x,foreign_gross,year,release_date,production_budget,domestic_gross_y,worldwide_gross
7926,Red 11,NaN,NaN,NaN,NaN,"Dec 31, 2018","$7,000",$0,$0
7927,Following,NaN,NaN,NaN,NaN,"Apr 2, 1999","$6,000","$48,482","$240,495"
7928,Return to the Land of Wonders,NaN,NaN,NaN,NaN,"Jul 13, 2005","$5,000","$1,338","$1,338"
7929,A Plague So Pleasant,NaN,NaN,NaN,NaN,"Sep 29, 2015","$1,400",$0,$0
7930,My Date With Drew,NaN,NaN,NaN,NaN,"Aug 5, 2005","$1,100","$181,041","$181,041"


In [22]:
#information about the merged dataframe
bom_and_tn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7931 entries, 0 to 7930
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie              7931 non-null   object 
 1   studio             3391 non-null   object 
 2   domestic_gross_x   3368 non-null   float64
 3   foreign_gross      2044 non-null   object 
 4   year               3396 non-null   float64
 5   release_date       5782 non-null   object 
 6   production_budget  5782 non-null   object 
 7   domestic_gross_y   5782 non-null   object 
 8   worldwide_gross    5782 non-null   object 
dtypes: float64(2), object(7)
memory usage: 619.6+ KB


In [23]:
bom_and_tn.isna().sum()

movie                   0
studio               4540
domestic_gross_x     4563
foreign_gross        5887
year                 4535
release_date         2149
production_budget    2149
domestic_gross_y     2149
worldwide_gross      2149
dtype: int64

In [24]:
#removing dollar signs and commas in this columns
bom_and_tn['production_budget'] = bom_and_tn['production_budget'].str.replace('[$,]', '').astype(float)
bom_and_tn['domestic_gross_y'] = bom_and_tn['domestic_gross_y'].str.replace('[$,]', '').astype(float)
bom_and_tn['worldwide_gross'] = bom_and_tn['worldwide_gross'].str.replace('[$,]', '').astype(float)

# fill missing values in 'domestic gross y' column with values from 'domestic gross x' column
bom_and_tn['domestic_gross_y'] = bom_and_tn['domestic_gross_y'].fillna(bom_and_tn['domestic_gross_x'])


In [25]:
bom_and_tn.tail()

,movie,studio,domestic_gross_x,foreign_gross,year,release_date,production_budget,domestic_gross_y,worldwide_gross
7926,Red 11,NaN,NaN,NaN,NaN,"Dec 31, 2018",7000.0,0.0,0.0
7927,Following,NaN,NaN,NaN,NaN,"Apr 2, 1999",6000.0,48482.0,240495.0
7928,Return to the Land of Wonders,NaN,NaN,NaN,NaN,"Jul 13, 2005",5000.0,1338.0,1338.0
7929,A Plague So Pleasant,NaN,NaN,NaN,NaN,"Sep 29, 2015",1400.0,0.0,0.0
7930,My Date With Drew,NaN,NaN,NaN,NaN,"Aug 5, 2005",1100.0,181041.0,181041.0


In [26]:
#changing to datetime data type and keeping the year only
bom_and_tn['release_date'] = pd.to_datetime(bom_and_tn['release_date']).dt.strftime('%Y')

In [27]:
#replacing nan with year from the column release date
bom_and_tn['year'] = bom_and_tn['year'].fillna(bom_and_tn['release_date'])

In [28]:
bom_and_tn.tail()

,movie,studio,domestic_gross_x,foreign_gross,year,release_date,production_budget,domestic_gross_y,worldwide_gross
7926,Red 11,NaN,NaN,NaN,2018,2018,7000.0,0.0,0.0
7927,Following,NaN,NaN,NaN,1999,1999,6000.0,48482.0,240495.0
7928,Return to the Land of Wonders,NaN,NaN,NaN,2005,2005,5000.0,1338.0,1338.0
7929,A Plague So Pleasant,NaN,NaN,NaN,2015,2015,1400.0,0.0,0.0
7930,My Date With Drew,NaN,NaN,NaN,2005,2005,1100.0,181041.0,181041.0


In [29]:
bom_and_tn.studio.value_counts().head(10)

IFC      166
Uni.     149
WB       141
Fox      137
Magn.    137
SPC      123
Sony     111
BV       106
LGF      103
Par.     102
Name: studio, dtype: int64

In [30]:
missing_percentage_studio = bom_and_tn['domestic_gross_y'].isna().sum() * 100 / len(bom_and_tn.domestic_gross_y)
missing_percentage_studio

0.3278275122935317

In [31]:
bom_and_tn["studio"] = np.where(bom_and_tn["studio"].isna(),
                           pd.Series([random.choice(["IFC", "Uni.", "WB", "Magn", "Fox", "SPC", "Sony", "BV", "LGF", "Par."])
                           for i in range(len(bom_and_tn))]),
                           bom_and_tn["studio"])

In [32]:
#specifying a two decimal place for floats
pd.options.display.float_format = '{:.2f}'.format

In [33]:
#removing the columns not needed
bom_and_tn = bom_and_tn.drop(columns=[ 'domestic_gross_x', 'release_date', 'foreign_gross'])

In [34]:
#renaming the column domestic_gross_y to domestic_gross
bom_and_tn =bom_and_tn.rename(columns={'domestic_gross_y': 'domestic_gross'})

#changing a column data type
bom_and_tn['year'] = bom_and_tn['year'].astype('int64')
bom_and_tn

,movie,studio,year,production_budget,domestic_gross,worldwide_gross
0,Toy Story 3,BV,2010,200000000.00,415004880.00,1068879522.00
1,Alice in Wonderland (2010),BV,2010,nan,334200000.00,nan
2,Harry Potter and the Deathly Hallows Part 1,WB,2010,nan,296000000.00,nan
3,Inception,WB,2010,160000000.00,292576195.00,835524642.00
4,Shrek Forever After,P/DW,2010,165000000.00,238736787.00,756244673.00
...,...,...,...,...,...,...
7926,Red 11,Uni.,2018,7000.00,0.00,0.00
7927,Following,Sony,1999,6000.00,48482.00,240495.00
7928,Return to the Land of Wonders,Fox,2005,5000.00,1338.00,1338.00
7929,A Plague So Pleasant,LGF,2015,1400.00,0.00,0.00


In [35]:
#creating a new column foreign gross
bom_and_tn['foreign_gross'] = (bom_and_tn['worldwide_gross'] - bom_and_tn['domestic_gross'])

In [36]:
bom_and_tn.tail()

,movie,studio,year,production_budget,domestic_gross,worldwide_gross,foreign_gross
7926,Red 11,Uni.,2018,7000.00,0.00,0.00,0.00
7927,Following,Sony,1999,6000.00,48482.00,240495.00,192013.00
7928,Return to the Land of Wonders,Fox,2005,5000.00,1338.00,1338.00,0.00
7929,A Plague So Pleasant,LGF,2015,1400.00,0.00,0.00,0.00
7930,My Date With Drew,Fox,2005,1100.00,181041.00,181041.00,0.00


In [37]:
bom_and_tn.isna().sum()

movie                   0
studio                  0
year                    0
production_budget    2149
domestic_gross         26
worldwide_gross      2149
foreign_gross        2149
dtype: int64

In [38]:
#calculating the percentage of NaN in domestic gross
percent_nan = bom_and_tn['domestic_gross'].isna().sum() / len(bom_and_tn.domestic_gross) * 100
percent_nan

0.3278275122935317

In [39]:
percent_nan_ww = (bom_and_tn['worldwide_gross'].isna().sum()) / len(bom_and_tn.worldwide_gross) * 100
percent_nan_ww

27.09620476610768

In [40]:
percent_nan_ww = (bom_and_tn['foreign_gross'].isna().sum()) / len(bom_and_tn.foreign_gross) * 100
percent_nan_ww

27.09620476610768

In [41]:
bom_and_tn = bom_and_tn.dropna(subset=['domestic_gross', 'production_budget', 'worldwide_gross'])

In [42]:
bom_and_tn

,movie,studio,year,production_budget,domestic_gross,worldwide_gross,foreign_gross
0,Toy Story 3,BV,2010,200000000.00,415004880.00,1068879522.00,653874642.00
3,Inception,WB,2010,160000000.00,292576195.00,835524642.00,542948447.00
4,Shrek Forever After,P/DW,2010,165000000.00,238736787.00,756244673.00,517507886.00
5,The Twilight Saga: Eclipse,Sum.,2010,68000000.00,300531751.00,706102828.00,405571077.00
6,Iron Man 2,Par.,2010,170000000.00,312433331.00,621156389.00,308723058.00
...,...,...,...,...,...,...,...
7926,Red 11,Uni.,2018,7000.00,0.00,0.00,0.00
7927,Following,Sony,1999,6000.00,48482.00,240495.00,192013.00
7928,Return to the Land of Wonders,Fox,2005,5000.00,1338.00,1338.00,0.00
7929,A Plague So Pleasant,LGF,2015,1400.00,0.00,0.00,0.00


In [43]:
bom_and_tn.isna().sum()

movie                0
studio               0
year                 0
production_budget    0
domestic_gross       0
worldwide_gross      0
foreign_gross        0
dtype: int64

In [44]:
imdb

,movie_id,movie,start_year,runtime_minutes,genres,averagerating,numvotes,region
0,tt0063540,Sunghursh,2013,175.00,"Action,Crime,Drama",7.00,77,IN
1,tt0066787,Ashad Ka Ek Din,2019,114.00,"Biography,Drama",7.20,43,"IN, XWW"
2,tt0069049,The Other Side of the Wind,2018,122.00,Drama,6.90,4517,"AR, BR, DE, ES, FR, GB, IT, PL, PT, RU, US, VE"
4,tt0100275,La Telenovela Errante,2017,80.00,"Comedy,Drama,Fantasy",6.50,119,"CL, PL, XWW"
6,tt0137204,Joe Finds Grace,2017,83.00,"Adventure,Animation,Comedy",8.10,263,CA
...,...,...,...,...,...,...,...,...
69547,tt9899860,Didan in film jorm ast,2019,100.00,"Drama,Thriller",8.10,7,"IR, XWW"
69548,tt9899880,Columbus,2018,85.00,Comedy,5.80,5,IR
69549,tt9903952,BADMEN with a good behavior,2018,87.00,"Comedy,Horror",9.20,5,DE
69550,tt9905462,Pengalila,2019,111.00,Drama,8.40,600,IN


In [45]:
bom_and_tn.head(5)

,movie,studio,year,production_budget,domestic_gross,worldwide_gross,foreign_gross
0,Toy Story 3,BV,2010,200000000.00,415004880.00,1068879522.00,653874642.00
3,Inception,WB,2010,160000000.00,292576195.00,835524642.00,542948447.00
4,Shrek Forever After,P/DW,2010,165000000.00,238736787.00,756244673.00,517507886.00
5,The Twilight Saga: Eclipse,Sum.,2010,68000000.00,300531751.00,706102828.00,405571077.00
6,Iron Man 2,Par.,2010,170000000.00,312433331.00,621156389.00,308723058.00


In [46]:
#merging bom gross and tn budgets
imdb_n_bom = pd.merge(bom_and_tn, imdb, on='movie', how='inner')

In [224]:
imdb_n_bom.head(3)

,movie,studio,year,production_budget,domestic_gross,worldwide_gross,foreign_gross,movie_id,start_year,runtime_minutes,genres,averagerating,numvotes,region
0,Toy Story 3,BV,2010,200000000.00,415004880.00,1068879522.00,653874642.00,tt0435761,2010,103.00,"Adventure,Animation,Comedy",8.30,682218,"AR, BG, BR, CA, CN, CZ, DK, EE, ES, FR, GE, GR..."
1,Inception,WB,2010,160000000.00,292576195.00,835524642.00,542948447.00,tt1375666,2010,148.00,"Action,Adventure,Sci-Fi",8.80,1841066,"AR, BE, BG, BR, CA, CL, CO, CZ, DE, DK, EE, ES..."
2,Shrek Forever After,P/DW,2010,165000000.00,238736787.00,756244673.00,517507886.00,tt0892791,2010,93.00,"Adventure,Animation,Comedy",6.30,167532,"AR, BE, BG, BR, CA, CM, CZ, DE, DK, EE, ES, FI..."
